In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np 
import math
from scipy.stats import ttest_ind, ttest_rel
pd.options.mode.chained_assignment = None  # default='warn'
import lmfit as lm
from scipy.stats import ttest_rel

SMALL_SIZE = 20
MEDIUM_SIZE = 24
BIGGER_SIZE = 28

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

print(plt.style.available)
plt.style.use('seaborn-white')

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [2]:
subjects = pd.read_csv("HCP/ndar_subject01.txt",delim_whitespace = True)

In [10]:
subjects[subjects["src_subject_id"]=="HCA9503576"]

,collection_id,ndar_subject01_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,race,ethnic_group,...,evaldate,multibirthid,tourettes_disorder,sample,race_other,respond,date_confirmed_phenotype,pos_years,position1,collection_title
582,2847,817400,37672,NDAR_INVUZ813VEE,HCA9503576,07/01/2017,819,F,Asian,Not Hispanic or Latino,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HCP-A Mapping the Human Connectome During Typi...


In [4]:
subjects.groupby("race").count()

,collection_id,ndar_subject01_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,ethnic_group,phenotype,...,evaldate,multibirthid,tourettes_disorder,sample,race_other,respond,date_confirmed_phenotype,pos_years,position1,collection_title
race,,,,,,,,,,,,,,,,,,,,,
American Indian/Alaska Native,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,2
Asian,52,52,52,52,52,52,52,52,52,52,...,0,0,0,0,0,0,0,0,0,52
Black or African American,101,101,101,101,101,101,101,101,101,101,...,0,0,0,0,0,0,0,0,0,101
More than one race,31,31,31,31,31,31,31,31,31,31,...,0,0,0,0,0,0,0,0,0,31
Race of study subject,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Unknown or not reported,14,14,14,14,14,14,14,14,14,14,...,0,0,0,0,0,0,0,0,0,14
White,525,525,525,525,525,525,525,525,525,525,...,0,0,0,0,0,0,0,0,0,525


In [15]:
for column in subjects.columns:
    print(column)

collection_id
ndar_subject01_id
dataset_id
subjectkey
src_subject_id
interview_date
interview_age
sex
race
ethnic_group
phenotype
phenotype_description
twins_study
sibling_study
family_study
family_user_def_id
subjectkey_mother
src_mother_id
subjectkey_father
src_father_id
subjectkey_sibling1
src_sibling1_id
sibling_type1
subjectkey_sibling2
src_sibling2_id
sibling_type2
subjectkey_sibling3
src_sibling3_id
sibling_type3
subjectkey_sibling4
src_sibling4_id
sibling_type4
zygosity
sample_taken
sample_id_original
sample_description
biorepository_name
sample_id_biorepository
patient_id_biorepository
cell_id_original
cell_id_biorepository
agre_subject_id
sfari_subject_id
cpea_site
cpea_id
blood_id
adi_dx
ados_dx
agp_family_id
agp_subject_id
site
comments_misc
week
study
reg_edu
reg_mother_edu
ca243
ccc2english
primary_language
physiological_responses_type
reg_father_edu
employ_status
bkgrnd_med
demo_logfinc
demo_wt_n_nr
demo_stratum
grade
service_specialed
asisped
brain_injury
trt_grp
versio